In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math



## Lecture 7
### 5 최대 예상손실액 구하기
> 최대 예상손실액 = VaR = Value at Risk
> - 위험관리를 위해 만들어진 개념

> VaR
> - 정상적인 시장 하에 일정기간 동안 발생할 수 있는 최대 손실금액

> 95% VaR=1,000만원
> - 해당 자산의 가치변동으로 인해 6개월동안 발생할 수 있는 최대 손실금액이 1000만원이 넘지 않을 확률이 95%
> - = 손실금액이 1000만원을 초과할 확률이 5%

> Q: 5%의 확률로 손실하는 1000만원 <- 이 값은 어케나온걸까?
> A: 과거 데이터로...
> - 삼전 과거 10년치를 봐서, 극단적인 상황이 나올 가능성이 가령 5%의 가능성으로 극단적인 - 나오는게 어느정도 인지 보는 것
> - 왼쪽 꼬리가 5% 정도인, - 수익률이 얼마인지 구하는 것

> 구하는 방법: boostrapping



- 6개월 간 보유수익률 = 125개의 daily return 모두 더하면 됨 -> 로그 수익률

> 로그수익률 쓰는 이유
> - 로그수익률: log(p_t)-log(p_(t-1))
> - 범위가 대칭적: -무한대 ~ +무한대
> - cf) 일반 수익률: -100% ~ +무한대
> - 가산성
> - 필참

In [4]:
df=pd.read_csv('c:/temp/samsung_LN7.csv',index_col='Date',parse_dates=True)
df1=df.loc['2012-01-01':'2021-12-31'] #10년동안 데이터로 편집
df1.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2012-01-02,21400,21600,21300,21600,263300,0.020794
2012-01-03,21860,22100,21840,22100,339046,0.023148
2012-01-04,22100,22200,21500,21600,342389,-0.022624
2012-01-05,21460,21580,21100,21100,346691,-0.023148
2012-01-06,21120,21319,20600,20800,376753,-0.014218


In [6]:
# 로그 수익률 변수 칼럼 만들기
# shift(1): 바로 앞의 값
df1['ret']=np.log(df1['Close'])-np.log(df1['Close'].shift(1))
df1.tail()

,Open,High,Low,Close,Volume,Change,ret
Date,,,,,,,
2021-12-24,80200,80800,80200,80500,12086380,0.007509,0.007481
2021-12-27,80600,80600,79800,80200,10783368,-0.003727,-0.003734
2021-12-28,80200,80400,79700,80300,18226325,0.001247,0.001246
2021-12-29,80200,80200,78500,78800,19794795,-0.018680,-0.018857
2021-12-30,78900,79500,78100,78300,14236700,-0.006345,-0.006365


In [7]:
df1.head()

,Open,High,Low,Close,Volume,Change,ret
Date,,,,,,,
2012-01-02,21400,21600,21300,21600,263300,0.020794,NaN
2012-01-03,21860,22100,21840,22100,339046,0.023148,0.022884
2012-01-04,22100,22200,21500,21600,342389,-0.022624,-0.022884
2012-01-05,21460,21580,21100,21100,346691,-0.023148,-0.023420
2012-01-06,21120,21319,20600,20800,376753,-0.014218,-0.014320


In [8]:
# ret: daily log return

In [9]:
# df2: df1 결측치를 삭제하자
df2=df1.dropna()
df2.head()

,Open,High,Low,Close,Volume,Change,ret
Date,,,,,,,
2012-01-03,21860,22100,21840,22100,339046,0.023148,0.022884
2012-01-04,22100,22200,21500,21600,342389,-0.022624,-0.022884
2012-01-05,21460,21580,21100,21100,346691,-0.023148,-0.023420
2012-01-06,21120,21319,20600,20800,376753,-0.014218,-0.014320
2012-01-09,20800,20820,20300,20320,387166,-0.023077,-0.023347


In [10]:
# ret만 가져오자
df3=df2['ret']
df3.tail()

Date
2021-12-24    0.007481
2021-12-27   -0.003734
2021-12-28    0.001246
2021-12-29   -0.018857
2021-12-30   -0.006365
Name: ret, dtype: float64

In [11]:
len(df3)

2462

In [12]:
# 10년간 로그 수익률 데이터가 2462개 있음

In [13]:
# 2462개 데이터 중에서 125개 꺼내서 더한다
# 이 작업을 여러번해서 분포를 그린다
# loop로 이 작업을 1000번 한다

In [20]:
sum_boots=[] # 1000개의 6개월 보유수익률이 sum_boots에 쌓인다
days=125
n=1000
random.seed(1234)
for i in range(n):
    sample=np.random.choice(df3,size=days,replace=True)
    sample_sum=sample.sum() # 뽑은 125개 로그수익률 모두 더한 값
    sum_boots.append(sample_sum)
len(sum_boots)

1000

In [17]:
# 1000번이야 10000번이야?
# 강의보기

In [21]:
sum_boots

[0.3572222722802003,
 -0.23633051142103412,
 -0.045272803793073635,
 0.4036084916795559,
 -0.14339680578480518,
 -0.017523170342556682,
 0.08682651583205647,
 0.1609155478734987,
 0.2867616819925676,
 -0.06443005126819656,
 0.1508233511825754,
 -0.005999134007588225,
 -0.3319427224634346,
 0.1655072387149623,
 -0.035241875082205,
 0.08614960390341331,
 -0.030224565682370752,
 -0.15112472544349487,
 0.14414662940407652,
 -0.09553219723629347,
 0.08000361327260563,
 0.32608126414289096,
 0.045842929083709905,
 -0.20864639512960537,
 0.18860223078825022,
 0.08926990685641556,
 0.20062476164533116,
 0.08901583761354281,
 0.260131994135941,
 0.21216057025655033,
 -0.054659828950754275,
 0.030951421890172526,
 0.017609153051216353,
 0.22552601107243042,
 0.23922356884324714,
 -0.00947585538383322,
 0.14421004901659273,
 0.02520032488981805,
 0.1838165224992956,
 0.27843060235638895,
 -0.05200364632168686,
 -0.1610246463382783,
 0.20745831764551248,
 0.26982787117153784,
 -0.07754316651004522

In [22]:
# 무작위로 뽑은 결과의 첫번째에 따르면 6개월 보유 수익률이 35%

In [23]:
# Q: 이 1000개의 값 중에서 5% 확률에 있는 마이너스 수익률은?
# A: np의 percentile 활용

In [25]:
ret5=np.percentile(sum_boots,5)
ret5

-0.21639086601900184

In [26]:
# 5%의 확률로 -21%의 수익률
# 최대손실금액: 1억*21%=2100만원 
# 95%의 VaR: 2100만원

In [28]:
# 90% VaR = 1600만원
ret6=np.percentile(sum_boots,10)
ret6

-0.16211650456792057

## Lecture 8 가구주 직업별 소득 차이(1)
### 1 가계소득 데이터: 가계금융복지조사

In [30]:
df=pd.read_excel('c:/temp/HOUSEHOLD2020.xlsx')
df.head()

,기준년도,MD제공용_가구고유번호,가중값,수도권여부,가구주_성별코드,가구원수,노인가구코드,조손가구코드,한부모가구코드,다문화가구(2012년~),...,원리금상환기인가계지출감소항목코드,부채상환가능시기코드,가구주_은퇴여부,가구주_미은퇴_은퇴연령,가구주_미은퇴_최소생활비,가구주_미은퇴_적정생활비,가구주_미은퇴_노후준비상황코드,가구주_은퇴실제연령,가구주_은퇴_적정생활비충당여부,가구주은퇴_생활비재원마련코드(2014년~)
0,2020,10000112,1531.602344,G1,1,3,G2,G2,G2,G2,...,NaN,1.0,1,70.0,400,500,3.0,NaN,NaN,NaN
1,2020,10000132,1314.119926,G1,2,2,G2,G2,G2,G2,...,4.0,1.0,1,70.0,500,700,3.0,NaN,NaN,NaN
2,2020,10000162,1098.806050,G1,2,1,G1,G2,G2,G2,...,NaN,NaN,1,75.0,100,150,3.0,NaN,NaN,NaN
3,2020,10000182,1124.027871,G1,1,2,G2,G2,G2,G2,...,NaN,NaN,1,70.0,200,250,4.0,NaN,NaN,NaN
4,2020,10000192,1098.806050,G1,2,1,G2,G2,G2,G2,...,NaN,NaN,1,65.0,250,300,3.0,NaN,NaN,NaN


In [31]:
# csv로는 한글이름 변수가 안불러와진다
# ,encoding='CP949' 넣으면 불러와짐

In [32]:
df_csv=pd.read_csv('c:/temp/HOUSEHOLD2020.csv',encoding='CP949')
df_csv.head()

,기준년도,MD제공용_가구고유번호,가중값,수도권여부,가구주_성별코드,가구원수,노인가구코드,조손가구코드,한부모가구코드,다문화가구(2012년~),...,원리금상환기인가계지출감소항목코드,부채상환가능시기코드,가구주_은퇴여부,가구주_미은퇴_은퇴연령,가구주_미은퇴_최소생활비,가구주_미은퇴_적정생활비,가구주_미은퇴_노후준비상황코드,가구주_은퇴실제연령,가구주_은퇴_적정생활비충당여부,가구주은퇴_생활비재원마련코드(2014년~)
0,2020,10000112,1531.602344,G1,1,3,G2,G2,G2,G2,...,NaN,1.0,1,70.0,400,500,3.0,NaN,NaN,NaN
1,2020,10000132,1314.119926,G1,2,2,G2,G2,G2,G2,...,4.0,1.0,1,70.0,500,700,3.0,NaN,NaN,NaN
2,2020,10000162,1098.806050,G1,2,1,G1,G2,G2,G2,...,NaN,NaN,1,75.0,100,150,3.0,NaN,NaN,NaN
3,2020,10000182,1124.027871,G1,1,2,G2,G2,G2,G2,...,NaN,NaN,1,70.0,200,250,4.0,NaN,NaN,NaN
4,2020,10000192,1098.806050,G1,2,1,G2,G2,G2,G2,...,NaN,NaN,1,65.0,250,300,3.0,NaN,NaN,NaN


In [33]:
df.columns # 칼럼 머있는지 보기

Index(['기준년도', 'MD제공용_가구고유번호', '가중값', '수도권여부', '가구주_성별코드', '가구원수', '노인가구코드',
       '조손가구코드', '한부모가구코드', '다문화가구(2012년~)',
       ...
       '원리금상환기인가계지출감소항목코드', '부채상환가능시기코드', '가구주_은퇴여부', '가구주_미은퇴_은퇴연령',
       '가구주_미은퇴_최소생활비', '가구주_미은퇴_적정생활비', '가구주_미은퇴_노후준비상황코드', '가구주_은퇴실제연령',
       '가구주_은퇴_적정생활비충당여부', '가구주은퇴_생활비재원마련코드(2014년~)'],
      dtype='object', length=153)

In [34]:
# 다 한국어로된 칼럼

In [37]:
columns=['기준년도','수도권여부','가구주_성별코드','가구원수','가구주_만연령','경상소득(보완)','가구주_직업대분류코드']

In [38]:
df1=df[columns]
df1.head()

,기준년도,수도권여부,가구주_성별코드,가구원수,가구주_만연령,경상소득(보완),가구주_직업대분류코드
0,2020,G1,1,3,34,6593,3
1,2020,G1,2,2,45,17720,2
2,2020,G1,2,1,73,908,4
3,2020,G1,1,2,58,2748,5
4,2020,G1,2,1,27,1015,2


In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18064 entries, 0 to 18063
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   기준년도         18064 non-null  int64 
 1   수도권여부        18064 non-null  object
 2   가구주_성별코드     18064 non-null  int64 
 3   가구원수         18064 non-null  int64 
 4   가구주_만연령      18064 non-null  int64 
 5   경상소득(보완)     18064 non-null  int64 
 6   가구주_직업대분류코드  13214 non-null  object
dtypes: int64(5), object(2)
memory usage: 988.0+ KB


In [40]:
# 문자변수(object) 2개: 수도권여부, 가구주_직업대분류코드